<h1>Capstone Project</h1>

<h2>Stock Prediction Using Twitter Sentiment Analysis</h2>

### Load the datatset

In [49]:
import pandas as pd

fname = open('finalData.csv', 'r')
df = pd.read_csv(fname)
df.head()

,S.no.,Date,Open,High,Low,Close,neg,pos
0,0,2009-06-18,8496.73,8590.52,8475.12,8555.60,0.6436,0.3564
1,1,2009-06-19,8556.96,8616.59,8496.73,8539.73,0.6369,0.3631
2,2,2009-06-20,8547.74,8577.71,8415.64,8439.37,0.6488,0.3512
3,3,2009-06-21,8543.13,8558.27,8375.09,8389.19,0.6476,0.3524
4,4,2009-06-22,8538.52,8538.83,8334.55,8339.01,0.6476,0.3524


In [50]:
'''
make the train and testing dataset.
'''
train_df = df[:int(len(df)*0.8)]
test_df = df[int(len(df)*0.8):]

In [54]:
train_df.loc[1]['Low']

8496.7299999999996

In [78]:
def build_set(t_df):
    count = len(t_df)
    data_row = {'Date':[], 'Open':[], 'High': [], 'Low':[], 'Close':[], 'neg': [], 'pos': [], 'target': []}
    for i in range(count-1):
        data_row['Date'].append(t_df.loc[i]['Date'])
        data_row['Open'].append(t_df.loc[i]['Open'])
        data_row['High'].append(t_df.loc[i]['High'])
        data_row['Low'].append(t_df.loc[i]['Low'])
        data_row['Close'].append(t_df.loc[i]['Close'])
        data_row['neg'].append(t_df.loc[i]['neg'])
        data_row['pos'].append(t_df.loc[i]['pos'])
        data_row['target'].append(t_df.loc[i+1]['Close'])
    
#     return data_row
    final_df = pd.DataFrame.from_dict(data=data_row,orient='columns', dtype=None)
    return final_df

In [80]:
data = build_set(train_df)
data.head()

,Close,Date,High,Low,Open,neg,pos,target
0,8555.60,2009-06-18,8590.52,8475.12,8496.73,0.6436,0.3564,8539.73
1,8539.73,2009-06-19,8616.59,8496.73,8556.96,0.6369,0.3631,8439.37
2,8439.37,2009-06-20,8577.71,8415.64,8547.74,0.6488,0.3512,8389.19
3,8389.19,2009-06-21,8558.27,8375.09,8543.13,0.6476,0.3524,8339.01
4,8339.01,2009-06-22,8538.83,8334.55,8538.52,0.6476,0.3524,8322.91
